In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from coco_stuff import COCOStuff
from pathlib import Path
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize, ToTensor, Resize, Lambda
from torch import LongTensor, FloatTensor
import torch
import PIL
import cv2
import numpy as np
import torch.nn.functional as F

In [ ]:
DATA = Path("data")
IMG = DATA / "images"
IMG_TRAIN = IMG / "train2017"
IMG_VAL = IMG / "val2017"

ANNOT = DATA / "annotations"
ANNOT_TRAIN = ANNOT / "stuff_train2017.json"
ANNOT_VAL = ANNOT / "stuff_val2017.json"

In [ ]:
INPUT_SIZE = (512, 512)

In [ ]:
image_transforms = [
    ToTensor(), 
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    Lambda(
        lambda img: F.interpolate(img.unsqueeze(0), INPUT_SIZE, mode="bilinear", align_corners=True).squeeze()
    )

]

mask_transforms = [
    Lambda(lambda mask: FloatTensor(np.expand_dims(mask, axis=2)).permute(2, 0, 1)),
    Lambda(
        lambda mask: F.interpolate(
            mask.unsqueeze(0), INPUT_SIZE, mode="nearest"
        ).squeeze()
    ),
    Lambda(lambda mask: mask.squeeze().long())    
]

In [ ]:
ds_train = COCOStuff(
    images_path=IMG_TRAIN,
    annotations_json=ANNOT_TRAIN,
    transformations=image_transforms,
    target_transformations=mask_transforms
)


ds_val = COCOStuff(
    images_path=IMG_VAL,
    annotations_json=ANNOT_VAL,
    transformations=image_transforms,
    target_transformations=mask_transforms
)


In [ ]:
ds_val.n_classes

In [ ]:
img, mask = ds_val.get_image_and_mask(ds_val._image_id(4145))

In [ ]:
plt.imshow(img)
plt.imshow(mask, alpha=0.7)
plt.show()

In [ ]:
np.unique(mask)